In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/My Drive/Research Project

/content/drive/My Drive/Research Project


In [3]:
import pandas as pd
import numpy as np
import os
# import src
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
pip install -U mittens

Requirement already up-to-date: mittens in /usr/local/lib/python3.6/dist-packages (0.2)


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from statistics import mean, stdev, median, mode
# With PCA
from sklearn.decomposition import PCA

from sklearn.feature_extraction.text import CountVectorizer
# SVM
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
# KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from mittens import GloVe, Mittens

In [6]:
# pip install autocorrect

In [7]:

import nltk
import string
from nltk.tokenize import word_tokenize
import random
import pickle
from nltk.corpus import stopwords

# from autocorrect import Speller
# from pycontractions import Contractions

# from spellchecker import SpellChecker

import re
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer 

from hyperopt import fmin, tpe, hp

In [8]:
nltk.download('punk')

[nltk_data] Error loading punk: Package 'punk' not found in index


False

In [9]:
from numpy import asarray
from numpy import zeros
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [10]:
# nltk.download('all')


## Loading Data

In [11]:
data_path = r'/content/drive/My Drive/Research Project/Data'

In [12]:
# data_path = r'C:\Users\kalya\OneDrive - University of Illinois at Chicago\!UIC\!Semesters\3rd Sem\CS 583 Data Mining and Text Mining\Research Project\Data'

In [13]:
train1_pr_df = pd.read_csv(os.path.join(data_path, 'Romney Training1 Data.csv'), usecols = [1,2])
# romney_train_pr_df = pd.read_csv(os.path.join(data_path, 'Romney Training1 Data.csv'), usecols = [1,2])

In [14]:
train1_pr_df.head()

,Doc Text,Sentiment
0,insidious mitt romney bain help philip morris ...,Negative
1,mean like romney cheat primary,Negative
2,mitt romney still believe black president,Negative
3,romney tax plan deserve nd look secret one dif...,Negative
4,hope romney debate prepped people last time,Positive


In [15]:
train1_pr_df = train1_pr_df.dropna()
# romney_train_pr_df = romney_train_pr_df.dropna()

In [16]:
train1_pr_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5647 entries, 0 to 5646
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Doc Text   5647 non-null   object
 1   Sentiment  5647 non-null   object
dtypes: object(2)
memory usage: 132.4+ KB


In [17]:
train1_pr_df.iloc[5573]

Doc Text     mitt romney talk lot thing fix tell massachuse...
Sentiment                                             Negative
Name: 5573, dtype: object

### Converting into TF-IDF

In [18]:
def tf_idf(x):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(x)
    #print(tfidf_matrix)
    x1 = tfidf_matrix.toarray()
#     print(x1)
    return x1

In [19]:
# train1_pr_tfidf = tf_idf(train1_pr_df['Doc Text'])
# # romney_train_pr_tfidf = tf_idf(romney_train_pr_df['Doc Text'])

In [20]:
# train1_pr_tfidf.shape

### Cross Valid Function

In [21]:
def cross_valid(X,y,model):
    f1_Positive  =[]
    f1_Neutral =[]
    f1_Negative =[]
    acc =[]
    cv = KFold(n_splits=5,shuffle=True)
    for train_index, val_index in cv.split(X):
    #     print("Train Index: ", train_index, "\n")
    #     print("Test Index: ", test_index)
    #     print(X.iloc[train_index])
    #     print(f)
        # print(1)
        X_train1, X_val, y_train1, y_val = X[train_index], X[val_index], y.iloc[train_index], y.iloc[val_index]
        temp = y_train1.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))

        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(X_train1)
        #print(tfidf_matrix)
        X_train1_tfidf = tfidf_matrix.toarray()

        model = model.fit(X_train1_tfidf, temp)

        tfidf_matrix2 = vectorizer.transform(X_val)
        #print(tfidf_matrix)
        X_val_tfidf = tfidf_matrix2.toarray()

        y_pred = model.predict(X_val_tfidf)
        temp_y_val = y_val.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
        f1_temp = f1_score(temp_y_val, y_pred, average = None)

        f1_Positive.append(f1_temp[0])
        f1_Neutral.append(f1_temp[1])
        f1_Negative.append(f1_temp[2])

        acc.append(round(accuracy_score(temp_y_val, y_pred),3))

    return round(mean(acc),3), round(mean(f1_Positive),3), round(mean(f1_Neutral),3), round(mean(f1_Negative),3)

### Cross Valid func with PCA

In [22]:
def cross_valid_PCA(X,y,model, n):
    f1_Positive  =[]
    f1_Neutral =[]
    f1_Negative =[]
    acc =[]
    cv = KFold(n_splits=5,shuffle=True)
    
    pca = PCA(n_components=n)
    
    
    for train_index, val_index in cv.split(X):
    #     print("Train Index: ", train_index, "\n")
    #     print("Test Index: ", test_index)
    #     print(X.iloc[train_index])
    #     print(f)
        # print(1)
        X_train1, X_val, y_train1, y_val = X[train_index], X[val_index], y.iloc[train_index], y.iloc[val_index]

        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(X_train1)
        #print(tfidf_matrix)
        X_train1_tfidf = tfidf_matrix.toarray()

        x_pca1 = pca.fit_transform(X_train1_tfidf)
        temp = y_train1.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))

        model = model.fit(x_pca1, temp)

        tfidf_matrix2 = vectorizer.transform(X_val)
        #print(tfidf_matrix)
        X_val_tfidf = tfidf_matrix2.toarray()
        
        x_pca_val = pca.transform(X_val_tfidf)
        y_pred = model.predict(x_pca_val)
        temp_y_val = y_val.apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
        f1_temp = f1_score(temp_y_val, y_pred, average = None)

        f1_Positive.append(f1_temp[0])
        f1_Neutral.append(f1_temp[1])
        f1_Negative.append(f1_temp[2])

        acc.append(round(accuracy_score(temp_y_val, y_pred),3))

    return round(mean(acc),3), round(mean(f1_Positive),3), round(mean(f1_Neutral),3), round(mean(f1_Negative),3)

### Logistic Regression

In [23]:
from sklearn.linear_model import LogisticRegression

In [24]:


# # clf = SVC(kernel="rbf", gamma=1, class_weight='balanced')
# #     error = 1-(cross_val_score(clf, x_pca1, temp, cv = 5, scoring = 'f1_macro'))
# #     f1 = mean(error) + stdev(error) 
# #print(f1)


# clf = LogisticRegression(class_weight='balanced')
# accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf) 
# eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
# eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
# # print([f1_list[2],f1_list[1],f1_list[0]])

# print(eval_dict)


In [25]:
# With Bayesian Optimization, and LR

def objective_func_LR(args):

    C = args['C']
    # penalty = args['penalty']
    # multi_class = args['multi_class']
    # l1_ratio = args['l1_ratio']

    print(C)
    clf = LogisticRegression(C= C,class_weight='balanced',n_jobs =-1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
    print(eval_dict)
    return -(eval_score)


space = {'C': hp.uniform('C', 0,2),
        # 'penalty': hp.choice('penalty', ['l1', 'l2']), 
        # 'multi_class': hp.choice('multi_class',['ovr', 'multinomial']),

        #  'l1_ratio' : hp.uniform('l1_ratio',0,1)
        }                        
                                
                                
best_classifier_LR = fmin(objective_func_LR, space, algo=tpe.suggest, max_evals=10)
print(best_classifier_LR)

0.9347097300313227
{'accuracy': 0.549, 'f1_pos': 0.487, 'f1_neu': 0.436, 'f1_neg': 0.649, 'eval_score': 0.499}
0.3784826973216604
{'accuracy': 0.55, 'f1_pos': 0.488, 'f1_neu': 0.446, 'f1_neg': 0.644, 'eval_score': 0.5}
1.2946974336058164
{'accuracy': 0.554, 'f1_pos': 0.477, 'f1_neu': 0.446, 'f1_neg': 0.653, 'eval_score': 0.492}
1.6161210048260575
{'accuracy': 0.553, 'f1_pos': 0.478, 'f1_neu': 0.443, 'f1_neg': 0.655, 'eval_score': 0.493}
0.6364944103054235
{'accuracy': 0.555, 'f1_pos': 0.493, 'f1_neu': 0.451, 'f1_neg': 0.649, 'eval_score': 0.505}
0.18207785963151157
{'accuracy': 0.546, 'f1_pos': 0.498, 'f1_neu': 0.433, 'f1_neg': 0.64, 'eval_score': 0.508}
0.43636591212831877
{'accuracy': 0.555, 'f1_pos': 0.496, 'f1_neu': 0.445, 'f1_neg': 0.651, 'eval_score': 0.508}
0.9500277761668727
{'accuracy': 0.551, 'f1_pos': 0.49, 'f1_neu': 0.439, 'f1_neg': 0.649, 'eval_score': 0.502}
0.45666859513840685
{'accuracy': 0.552, 'f1_pos': 0.492, 'f1_neu': 0.44, 'f1_neg': 0.65, 'eval_score': 0.504}
0.331

In [26]:
{'C': 0.18
{'accuracy': 0.546, 'f1_pos': 0.498, 'f1_neu': 0.433, 'f1_neg': 0.64, 'eval_score': 0.508}

{'accuracy': 0.59,
 'eval_score': 0.608,
 'f1_neg': 0.621,
 'f1_neu': 0.535,
 'f1_pos': 0.613}

In [27]:
# With PCA, Bayesian Optimization, and LR

def objective_func_LR_PCA(args):

    C = args['C']
    n = args['n']
    
    # penalty = args['penalty']
    # multi_class = args['multi_class']
    # l1_ratio = args['l1_ratio']


    clf = LogisticRegression(C= C,class_weight='balanced',n_jobs =-1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)
    par_dict = {'C': C, 'n': n}
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
    print(par_dict)
    print(eval_dict)
    print('\n')    
    return -(eval_score)


space = {'C': hp.uniform('C', 0,2),
         'n': hp.choice('n', np.arange(100, 500, step =50))
        # 'penalty': hp.choice('penalty', ['l1', 'l2']), 
        # 'multi_class': hp.choice('multi_class',['ovr', 'multinomial']),
        #  'l1_ratio' : hp.uniform('l1_ratio',0,1)
        }                        
                                
                                
best_classifier_LR_PCA = fmin(objective_func_LR_PCA, space, algo=tpe.suggest, max_evals=20)


{'C': 0.534788843113817, 'n': 450}
{'accuracy': 0.539, 'f1_pos': 0.495, 'f1_neu': 0.439, 'f1_neg': 0.627, 'eval_score': 0.504}
{'C': 0.918976658694469, 'n': 300}
{'accuracy': 0.526, 'f1_pos': 0.481, 'f1_neu': 0.436, 'f1_neg': 0.61, 'eval_score': 0.49}
{'C': 0.8813766437703958, 'n': 300}
{'accuracy': 0.534, 'f1_pos': 0.493, 'f1_neu': 0.448, 'f1_neg': 0.613, 'eval_score': 0.501}
{'C': 1.3022399377701697, 'n': 150}
{'accuracy': 0.514, 'f1_pos': 0.471, 'f1_neu': 0.428, 'f1_neg': 0.595, 'eval_score': 0.48}
{'C': 0.9370661925495203, 'n': 300}
{'accuracy': 0.527, 'f1_pos': 0.477, 'f1_neu': 0.432, 'f1_neg': 0.616, 'eval_score': 0.487}
{'C': 1.897433594124727, 'n': 350}
{'accuracy': 0.53, 'f1_pos': 0.474, 'f1_neu': 0.44, 'f1_neg': 0.618, 'eval_score': 0.485}
{'C': 0.046094688226361535, 'n': 300}
{'accuracy': 0.518, 'f1_pos': 0.466, 'f1_neu': 0.404, 'f1_neg': 0.617, 'eval_score': 0.476}
{'C': 1.2744370420259483, 'n': 300}
{'accuracy': 0.522, 'f1_pos': 0.476, 'f1_neu': 0.423, 'f1_neg': 0.61, 'eva

In [28]:
{'C': 0.534788843113817, 'n': 450}
{'accuracy': 0.539, 'f1_pos': 0.495, 'f1_neu': 0.439, 'f1_neg': 0.627, 'eval_score': 0.504}

{'accuracy': 0.58,
 'eval_score': 0.592,
 'f1_neg': 0.612,
 'f1_neu': 0.53,
 'f1_pos': 0.595}

In [29]:
# # With Bayesian Optimization, different class weights and LR

# def objective_func_LR(args):

#     C = args['C']
#     balance = args['balance']
#     # multi_class = args['multi_class']
#     # l1_ratio = args['l1_ratio']

 
#     clf = LogisticRegression(C= C,class_weight=balance,n_jobs =-1)
#     accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf) 
#     eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
#     eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
#     print(eval_dict)
#     return -(eval_score)


# space = {'C': hp.uniform('C', 0,10),
#          'balance': hp.choice('balance', [{1:3, 2:1, 3:2}, {1:15, 2:1, 3:10}, {1:20, 2:5, 3:10}, {1:100, 2:10, 3:100}])
#         # 'penalty': hp.choice('penalty', ['l1', 'l2']), 
#         # 'multi_class': hp.choice('multi_class',['ovr', 'multinomial']),

#         #  'l1_ratio' : hp.uniform('l1_ratio',0,1)
#         }                        
                                
                                
# best_classifier_LR = fmin(objective_func_LR, space, algo=tpe.suggest, max_evals=20)
# print(best_classifier_LR)

### SVM

In [30]:
# clf = SVC(kernel = 'rbf', class_weight='balanced')
# accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf) 
# eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)

# eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
# print(eval_dict)

In [31]:
# # With Bayesian Optimization, and SVM

# def objective_func_SVM(args):
#     C = args['C']
# #     kernel = args['kernel']
# #     gamma = args['gamma']
# #     degree = args['degree']

#     par_dict = {'C': C}
#     print(par_dict)
 
#     clf = SVC(C= C, kernel = 'rbf', class_weight='balanced')
#     accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train1_pr_tfidf,train1_pr_df['Sentiment'], clf) 
#     eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)

#     eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
#     print(eval_dict)
#     print('\n')    
#     return -(eval_score)


# space = {'C': hp.uniform('C', 0,20)
# #         'kernel': hp.choice('kernel', ['poly', 'rbf']) 
# #         'gamma': hp.choice('gamma',range(1,4)),
# #          'degree' : hp.choice('degree',range(1,4))
#         }
                                
                                
                                
# best_classifier_SVM_PCA = fmin(objective_func_SVM, space, algo=tpe.suggest, max_evals=20)
# print(best_classifier_SVM_PCA)

In [32]:
# With PCA, Bayesian Optimization, and SVM

def objective_func_SVM_PCA(args):
    C = args['C']
    n = args['n']
#     kernel = args['kernel']
#     gamma = args['gamma']
#     degree = args['degree']

    par_dict = {'C': C, 'n': n, 'kernel': 'rbf'}
    print(par_dict)
 
    clf = SVC(C= C, kernel = 'rbf', class_weight='balanced')
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)

    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
    print(eval_dict)
    print('\n')    
    return -(eval_score)


space = {'C': hp.uniform('C', 0,5),
         'n': hp.choice('n', np.arange(3, 1003, step =100))
#         'kernel': hp.choice('kernel', ['poly', 'rbf']) 
#         'gamma': hp.choice('gamma',range(1,4)),
#          'degree' : hp.choice('degree',range(1,4))
        }
                                
                                
                                
best_classifier_SVM_PCA = fmin(objective_func_SVM_PCA, space, algo=tpe.suggest, max_evals=20)
print(best_classifier_SVM_PCA)

{'C': 4.846334348398816, 'n': 603, 'kernel': 'rbf'}
{'accuracy': 0.564, 'f1_pos': 0.456, 'f1_neu': 0.415, 'f1_neg': 0.68, 'eval_score': 0.478}
{'C': 2.461240924272388, 'n': 803, 'kernel': 'rbf'}
{'accuracy': 0.563, 'f1_pos': 0.472, 'f1_neu': 0.43, 'f1_neg': 0.671, 'eval_score': 0.49}
{'C': 1.2390471912399093, 'n': 403, 'kernel': 'rbf'}
{'accuracy': 0.554, 'f1_pos': 0.484, 'f1_neu': 0.447, 'f1_neg': 0.652, 'eval_score': 0.498}
{'C': 4.721511214430499, 'n': 803, 'kernel': 'rbf'}
{'accuracy': 0.558, 'f1_pos': 0.448, 'f1_neu': 0.41, 'f1_neg': 0.675, 'eval_score': 0.47}
{'C': 3.7046452603946007, 'n': 603, 'kernel': 'rbf'}
{'accuracy': 0.562, 'f1_pos': 0.457, 'f1_neu': 0.423, 'f1_neg': 0.675, 'eval_score': 0.478}
{'C': 1.530692401204854, 'n': 803, 'kernel': 'rbf'}
{'accuracy': 0.561, 'f1_pos': 0.493, 'f1_neu': 0.442, 'f1_neg': 0.661, 'eval_score': 0.507}
{'C': 2.797443607237336, 'n': 403, 'kernel': 'rbf'}
{'accuracy': 0.564, 'f1_pos': 0.466, 'f1_neu': 0.437, 'f1_neg': 0.675, 'eval_score': 0.

In [33]:
'C': 1.530692401204854, 'n': 803, 'kernel': 'rbf'}
{'accuracy': 0.561, 'f1_pos': 0.493, 'f1_neu': 0.442, 'f1_neg': 0.661, 'eval_score': 0.507}

{'accuracy': 0.59,
 'eval_score': 0.599,
 'f1_neg': 0.605,
 'f1_neu': 0.568,
 'f1_pos': 0.601}

### KNN

In [34]:
# WIth Bayesian Optimization and KNN
def objective_func_KNN(args):
    n_neighbors = args['n_neighbors']
    metric = args['metric']  

    par_dict = {'n_neighbors': n_neighbors, 'metric': metric}
    print(par_dict)
    
    clf = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)

    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_neighbors': hp.choice('n_neighbors',np.arange(1,31, step =3)),
        'metric':hp.choice('metric', ["euclidean","manhattan"])
        }

best_classifier_KNN = fmin(objective_func_KNN, space, algo=tpe.suggest, max_evals=10)
print(best_classifier_KNN)

{'n_neighbors': 25, 'metric': 'euclidean'}
{'accuracy': 0.551, 'f1_pos': 0.418, 'f1_neu': 0.344, 'f1_neg': 0.68, 'eval_score': 0.445}
{'n_neighbors': 19, 'metric': 'manhattan'}
{'accuracy': 0.406, 'f1_pos': 0.38, 'f1_neu': 0.356, 'f1_neg': 0.452, 'eval_score': 0.385}
{'n_neighbors': 16, 'metric': 'manhattan'}
{'accuracy': 0.39, 'f1_pos': 0.381, 'f1_neu': 0.354, 'f1_neg': 0.419, 'eval_score': 0.383}
{'n_neighbors': 16, 'metric': 'euclidean'}
{'accuracy': 0.53, 'f1_pos': 0.432, 'f1_neu': 0.36, 'f1_neg': 0.648, 'eval_score': 0.452}
{'n_neighbors': 4, 'metric': 'manhattan'}
{'accuracy': 0.417, 'f1_pos': 0.369, 'f1_neu': 0.41, 'f1_neg': 0.457, 'eval_score': 0.379}
{'n_neighbors': 28, 'metric': 'euclidean'}
{'accuracy': 0.543, 'f1_pos': 0.409, 'f1_neu': 0.317, 'f1_neg': 0.674, 'eval_score': 0.436}
{'n_neighbors': 25, 'metric': 'euclidean'}
{'accuracy': 0.543, 'f1_pos': 0.415, 'f1_neu': 0.331, 'f1_neg': 0.676, 'eval_score': 0.441}
{'n_neighbors': 22, 'metric': 'euclidean'}
{'accuracy': 0.538,

In [35]:
# WIth PCA , Bayesian Optimization and KNN
def objective_func_KNN_PCA(args):
    n_neighbors = args['n_neighbors']
    metric = args['metric']
    n = args['n']    

    par_dict = {'n_neighbors': n_neighbors, 'n': n, 'metric': metric}
    print(par_dict)
    
    clf = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)

    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_neighbors': hp.choice('n_neighbors',np.arange(1,10, step =1)),
        'metric':hp.choice('metric', ["euclidean","manhattan"]),
        'n': hp.choice('n', np.arange(3,63, step =5))}

best_classifier_KNN_PCA = fmin(objective_func_KNN_PCA, space, algo=tpe.suggest, max_evals=20)


{'n_neighbors': 9, 'n': 3, 'metric': 'euclidean'}
{'accuracy': 0.466, 'f1_pos': 0.222, 'f1_neu': 0.335, 'f1_neg': 0.601, 'eval_score': 0.271}
{'n_neighbors': 7, 'n': 38, 'metric': 'euclidean'}
{'accuracy': 0.508, 'f1_pos': 0.352, 'f1_neu': 0.386, 'f1_neg': 0.623, 'eval_score': 0.383}
{'n_neighbors': 2, 'n': 38, 'metric': 'euclidean'}
{'accuracy': 0.418, 'f1_pos': 0.361, 'f1_neu': 0.404, 'f1_neg': 0.466, 'eval_score': 0.372}
{'n_neighbors': 3, 'n': 58, 'metric': 'manhattan'}
{'accuracy': 0.489, 'f1_pos': 0.352, 'f1_neu': 0.382, 'f1_neg': 0.609, 'eval_score': 0.379}
{'n_neighbors': 5, 'n': 23, 'metric': 'manhattan'}
{'accuracy': 0.496, 'f1_pos': 0.347, 'f1_neu': 0.374, 'f1_neg': 0.621, 'eval_score': 0.377}
{'n_neighbors': 1, 'n': 18, 'metric': 'euclidean'}
{'accuracy': 0.472, 'f1_pos': 0.321, 'f1_neu': 0.365, 'f1_neg': 0.592, 'eval_score': 0.351}
{'n_neighbors': 3, 'n': 58, 'metric': 'manhattan'}
{'accuracy': 0.48, 'f1_pos': 0.365, 'f1_neu': 0.36, 'f1_neg': 0.598, 'eval_score': 0.388}
{'

### Naive Bayes

In [36]:
clf = GaussianNB()
accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf) 
eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}
print(eval_dict)

{'accuracy': 0.4, 'f1_pos': 0.369, 'f1_neu': 0.364, 'f1_neg': 0.449, 'eval_score': 0.406}


In [37]:
# With PCA and Naive bayes
def objective_func_NB_PCA(args):
    n = args['n']

    par_dict = {'n': n}
    print(par_dict)
    
    clf = GaussianNB()
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n': hp.choice('n', np.arange(3,63, step =5))}

best_classifier_NB_PCA = fmin(objective_func_NB_PCA, space, algo=tpe.suggest, max_evals=10)



{'n': 53}
{'accuracy': 0.446, 'f1_pos': 0.354, 'f1_neu': 0.436, 'f1_neg': 0.496, 'eval_score': 0.372}
{'n': 38}
{'accuracy': 0.434, 'f1_pos': 0.328, 'f1_neu': 0.417, 'f1_neg': 0.492, 'eval_score': 0.349}
{'n': 33}
{'accuracy': 0.451, 'f1_pos': 0.348, 'f1_neu': 0.39, 'f1_neg': 0.527, 'eval_score': 0.369}
{'n': 18}
{'accuracy': 0.448, 'f1_pos': 0.302, 'f1_neu': 0.38, 'f1_neg': 0.532, 'eval_score': 0.331}
{'n': 3}
{'accuracy': 0.492, 'f1_pos': 0.077, 'f1_neu': 0.16, 'f1_neg': 0.653, 'eval_score': 0.16}
{'n': 43}
{'accuracy': 0.444, 'f1_pos': 0.343, 'f1_neu': 0.438, 'f1_neg': 0.491, 'eval_score': 0.363}
{'n': 23}
{'accuracy': 0.453, 'f1_pos': 0.334, 'f1_neu': 0.388, 'f1_neg': 0.539, 'eval_score': 0.358}
{'n': 58}
{'accuracy': 0.435, 'f1_pos': 0.343, 'f1_neu': 0.438, 'f1_neg': 0.472, 'eval_score': 0.361}
{'n': 3}
{'accuracy': 0.497, 'f1_pos': 0.129, 'f1_neu': 0.177, 'f1_neg': 0.651, 'eval_score': 0.203}
{'n': 8}
{'accuracy': 0.482, 'f1_pos': 0.255, 'f1_neu': 0.191, 'f1_neg': 0.631, 'eval_sc

## Ensemble Methods

### Random Forest

In [38]:
# With Bayesian Optimization, and Random Forest

def objective_func_RF(args):
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    min_samples_split = args['min_samples_split']
    min_samples_leaf = args['min_samples_leaf']

    par_dict = {'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
    print(par_dict)
    
    clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
         'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
#         'criterion': hp.choice('criterion',['gini', 'entropy']),
#         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
        'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
         'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1))
        }
                                
                                
                                
best_classifier_RF = fmin(objective_func_RF, space, algo=tpe.suggest, max_evals=25)
print(best_classifier_RF)

{'n_estimators': 68, 'max_depth': 33, 'min_samples_split': 4, 'min_samples_leaf': 2}
{'accuracy': 0.535, 'f1_pos': 0.14, 'f1_neu': 0.096, 'f1_neg': 0.688, 'eval_score': 0.219}
{'n_estimators': 49, 'max_depth': 77, 'min_samples_split': 2, 'min_samples_leaf': 6}
{'accuracy': 0.556, 'f1_pos': 0.239, 'f1_neu': 0.24, 'f1_neg': 0.696, 'eval_score': 0.302}
{'n_estimators': 92, 'max_depth': 88, 'min_samples_split': 2, 'min_samples_leaf': 3}
{'accuracy': 0.569, 'f1_pos': 0.296, 'f1_neu': 0.264, 'f1_neg': 0.706, 'eval_score': 0.351}
{'n_estimators': 37, 'max_depth': 41, 'min_samples_split': 10, 'min_samples_leaf': 9}
{'accuracy': 0.536, 'f1_pos': 0.147, 'f1_neu': 0.101, 'f1_neg': 0.688, 'eval_score': 0.225}
{'n_estimators': 54, 'max_depth': 57, 'min_samples_split': 5, 'min_samples_leaf': 2}
{'accuracy': 0.552, 'f1_pos': 0.229, 'f1_neu': 0.183, 'f1_neg': 0.696, 'eval_score': 0.294}
{'n_estimators': 34, 'max_depth': 23, 'min_samples_split': 9, 'min_samples_leaf': 7}
{'accuracy': 0.53, 'f1_pos': 0.

In [39]:
# With PCA,  Bayesian Optimization, and Random Forest

def objective_func_RF_PCA(args):
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    min_samples_split = args['min_samples_split']
    min_samples_leaf = args['min_samples_leaf']
    n = args['n']

    par_dict = { 'n': n, 'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
    print(par_dict)
    
    clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
         'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
#         'criterion': hp.choice('criterion',['gini', 'entropy']),
#         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
        'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
         'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1)),
        'n': hp.choice('n', np.arange(3,203, step =20))
        }
                                
                                
                                
best_classifier_RF_PCA = fmin(objective_func_RF_PCA, space, algo=tpe.suggest, max_evals=30)
print(best_classifier_RF_PCA)

{'n': 43, 'n_estimators': 26, 'max_depth': 26, 'min_samples_split': 7, 'min_samples_leaf': 11}
{'accuracy': 0.547, 'f1_pos': 0.264, 'f1_neu': 0.278, 'f1_neg': 0.687, 'eval_score': 0.321}
{'n': 143, 'n_estimators': 44, 'max_depth': 62, 'min_samples_split': 9, 'min_samples_leaf': 1}
{'accuracy': 0.557, 'f1_pos': 0.276, 'f1_neu': 0.294, 'f1_neg': 0.696, 'eval_score': 0.332}
{'n': 23, 'n_estimators': 64, 'max_depth': 84, 'min_samples_split': 9, 'min_samples_leaf': 11}
{'accuracy': 0.557, 'f1_pos': 0.258, 'f1_neu': 0.293, 'f1_neg': 0.696, 'eval_score': 0.318}
{'n': 43, 'n_estimators': 70, 'max_depth': 34, 'min_samples_split': 5, 'min_samples_leaf': 2}
{'accuracy': 0.559, 'f1_pos': 0.319, 'f1_neu': 0.335, 'f1_neg': 0.69, 'eval_score': 0.367}
{'n': 43, 'n_estimators': 79, 'max_depth': 79, 'min_samples_split': 5, 'min_samples_leaf': 1}
{'accuracy': 0.559, 'f1_pos': 0.323, 'f1_neu': 0.328, 'f1_neg': 0.691, 'eval_score': 0.37}
{'n': 3, 'n_estimators': 78, 'max_depth': 56, 'min_samples_split': 10

In [40]:
# # WITH LDA
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from math import sqrt
# lda = LinearDiscriminantAnalysis(n_components = None)
# x_lda = lda.fit_transform(obama_train_pr_tfidf, obama_train_pr_df['Sentiment'])

In [41]:
# # With LDA, Bayesian Optimization, and Random Forest

# def objective_func(args):
#     n_estimators = args['n_estimators']
#     max_depth = args['max_depth']
#     min_samples_split = args['min_samples_split']
#     min_samples_leaf = args['min_samples_leaf']
# #     pca = PCA(n_components=args['n'])
# #     x_pca1 = pca.fit_transform(obama_train_pr_tfidf)

 
#     clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    
#     temp = obama_train_pr_df['Sentiment'].apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
#     temp_f1 = cross_val_score(clf, x_lda, temp, cv = 5, scoring = 'f1_macro')
#     f1 = mean(temp_f1)
#     return -(f1)
# space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
#          'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
# #         'criterion': hp.choice('criterion',['gini', 'entropy']),
# #         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
#         'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
#          'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1))
# #         'n': hp.choice('n', np.arange(3,20, step =1))
#         }
                                
                                
                                
# best_classifier = fmin(objective_func, space, algo=tpe.suggest, max_evals=30)
# print(best_classifier)# With Bayesian Optimization, and Random Forest



In [42]:
# obama_train2_pr_tfidf, obama_val_pr_tfidf, obama_train2_pr_class, obama_val_pr_class = train_test_split(obama_train_pr_tfidf, obama_train_pr_df['Sentiment'], test_size = 0.2)

In [43]:
# x_lda2 = lda.fit_transform(obama_train2_pr_tfidf, obama_train2_pr_class)

In [44]:
# # With LDA, Bayesian Optimization, and Random Forest

# def objective_func(args):
#     n_estimators = args['n_estimators']
#     max_depth = args['max_depth']
#     min_samples_split = args['min_samples_split']
#     min_samples_leaf = args['min_samples_leaf']
# #     pca = PCA(n_components=args['n'])
# #     x_pca1 = pca.fit_transform(obama_train_pr_tfidf)

 
#     clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    
# #     temp = obama_train_pr_df['Sentiment'].apply(lambda x: 1 if x == 'Positive' else (3 if x == 'Negative' else 2))
#     temp_f1 = cross_val_score(clf, x_lda2, obama_train2_pr_class, cv = 5, scoring = 'f1_macro')
#     f1 = mean(temp_f1)
#     return -(f1)
# space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
#          'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
# #         'criterion': hp.choice('criterion',['gini', 'entropy']),
# #         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
#         'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
#          'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1))
# #         'n': hp.choice('n', np.arange(3,20, step =1))
#         }
                                
                                
                                
# best_classifier = fmin(objective_func, space, algo=tpe.suggest, max_evals=30)
# print(best_classifier)# With Bayesian Optimization, and Random Forest



In [45]:
# x_val_lda = lda.transform(obama_val_pr_tfidf)
# x_val_lda_df = pd.DataFrame(data = x_val_lda)

In [46]:
# c = lambda b : 1 if b == 0 else b
# d = lambda b : 2 if b <= 1 else b
# bc = RandomForestClassifier(max_depth = c(best_classifier['max_depth']), min_samples_leaf = c(best_classifier['min_samples_leaf']), min_samples_split = d(best_classifier['min_samples_split'])
#                             ,n_estimators = best_classifier['n_estimators'])

# bc.fit(x_lda2, obama_train2_pr_class)

# y_pred = bc.predict(x_val_lda)

# accuracy_score(obama_val_pr_class, y_pred)

In [47]:
# lda_clf = lda.fit(obama_train2_pr_tfidf, obama_train2_pr_class)
# y_pred_lda_clf = lda_clf.predict(obama_val_pr_tfidf)
# # y_pred_lda_clf

# accuracy_score(obama_val_pr_class, y_pred_lda_clf)

### Light Gradient Boosting

In [48]:
import lightgbm as lgbm
from lightgbm.sklearn import LGBMClassifier

In [49]:
# With Bayesian Optimization, and Light GB

def objective_func_LGB(args):
    boosting_type= args['boosting_type']
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    learning_rate = args['learning_rate']
    subsample = args['subsample']
    min_split_gain = args['min_split_gain']

    par_dict = {'boosting_type': boosting_type, 'n_estimators': n_estimators, 'max_depth': max_depth, 'learning_rate': learning_rate, 'subsample': subsample, 'min_split_gain': min_split_gain}
    print(par_dict)
    
    clf = LGBMClassifier(boosting_type = boosting_type, class_weight  = 'balanced', num_leaves = 2**max_depth, n_estimators = n_estimators, max_depth = max_depth, subsample=subsample, learning_rate=learning_rate, min_split_gain = min_split_gain)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {
    'boosting_type' : hp.choice('boosting_type', ['gbdt', 'dart', 'goss']),
    'n_estimators': hp.choice('n_estimators', np.arange(50,501, step =1)),
    'learning_rate':  hp.uniform('learning_rate', 0,0.2), 
    'max_depth': hp.choice('max_depth', np.arange(3, 10, step =1)),   
     'subsample': hp.choice('subsample', np.linspace(0.8, 1, num=4)),
    'min_split_gain': hp.choice('min_split_gain' , np.arange(0,6, step =1))}
                                
                                
                                
best_classifier_LGB = fmin(objective_func_LGB, space, algo=tpe.suggest, max_evals=25)
print(best_classifier_LGB)

{'boosting_type': 'dart', 'n_estimators': 227, 'max_depth': 6, 'learning_rate': 0.0511984426499706, 'subsample': 0.8, 'min_split_gain': 2}
{'accuracy': 0.483, 'f1_pos': 0.417, 'f1_neu': 0.462, 'f1_neg': 0.534, 'eval_score': 0.43}
{'boosting_type': 'dart', 'n_estimators': 386, 'max_depth': 6, 'learning_rate': 0.07485468504996344, 'subsample': 0.8, 'min_split_gain': 2}
{'accuracy': 0.499, 'f1_pos': 0.427, 'f1_neu': 0.456, 'f1_neg': 0.57, 'eval_score': 0.441}
{'boosting_type': 'dart', 'n_estimators': 338, 'max_depth': 9, 'learning_rate': 0.18187732771394813, 'subsample': 0.8, 'min_split_gain': 0}
{'accuracy': 0.509, 'f1_pos': 0.438, 'f1_neu': 0.438, 'f1_neg': 0.596, 'eval_score': 0.452}
{'boosting_type': 'goss', 'n_estimators': 103, 'max_depth': 5, 'learning_rate': 0.15977778478262253, 'subsample': 0.9333333333333333, 'min_split_gain': 0}
{'accuracy': 0.476, 'f1_pos': 0.41, 'f1_neu': 0.409, 'f1_neg': 0.556, 'eval_score': 0.423}
{'boosting_type': 'dart', 'n_estimators': 483, 'max_depth': 9

In [50]:
# With PCA, Bayesian Optimization, and Light GB

def objective_func_LGB_PCA(args):
    boosting_type= args['boosting_type']
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    learning_rate = args['learning_rate']
#     subsample = args['subsample']
    min_split_gain = args['min_split_gain']
    n = args['n']
    
    par_dict = {'n': n, 'n_estimators': n_estimators, 'max_depth': max_depth, 'learning_rate': learning_rate, 'subsample': 1, 'min_split_gain': min_split_gain} 
    print(par_dict)
    
    clf = LGBMClassifier(boosting_type = boosting_type, class_weight  = 'balanced', num_leaves = 2**max_depth, n_estimators = n_estimators, max_depth = max_depth, subsample=1, learning_rate=learning_rate, min_split_gain = min_split_gain)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(train1_pr_df['Doc Text'],train1_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([0.4*accuracy, 1.6*f1_Positive]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {
    'boosting_type' : hp.choice('boosting_type', ['gbdt', 'dart', 'goss']),
    'n_estimators': hp.choice('n_estimators', np.arange(50,501, step =1)),
    'learning_rate':  hp.uniform('learning_rate', 0,0.2), 
    'max_depth': hp.choice('max_depth', np.arange(3, 10, step =1)),   
#      'subsample': hp.choice('subsample', np.linspace(0.8, 1, num=4)),
    'min_split_gain': hp.choice('min_split_gain' , np.arange(0,6, step =1)),
        'n': hp.choice('n', np.arange(3,203, step =20))}
                                
                                
                                
best_classifier_LGB_PCA = fmin(objective_func_LGB_PCA, space, algo=tpe.suggest, max_evals=35)
print(best_classifier_LGB_PCA)

{'n': 3, 'n_estimators': 302, 'max_depth': 4, 'learning_rate': 0.10168599580359312, 'subsample': 1, 'min_split_gain': 1}
{'accuracy': 0.401, 'f1_pos': 0.289, 'f1_neu': 0.35, 'f1_neg': 0.495, 'eval_score': 0.311}
{'n': 123, 'n_estimators': 398, 'max_depth': 4, 'learning_rate': 0.10743376165164781, 'subsample': 1, 'min_split_gain': 3}
{'accuracy': 0.504, 'f1_pos': 0.436, 'f1_neu': 0.42, 'f1_neg': 0.593, 'eval_score': 0.45}
{'n': 83, 'n_estimators': 291, 'max_depth': 5, 'learning_rate': 0.11063747943949197, 'subsample': 1, 'min_split_gain': 1}
{'accuracy': 0.526, 'f1_pos': 0.441, 'f1_neu': 0.43, 'f1_neg': 0.623, 'eval_score': 0.458}
{'n': 43, 'n_estimators': 311, 'max_depth': 5, 'learning_rate': 0.040485855950508645, 'subsample': 1, 'min_split_gain': 2}
{'accuracy': 0.498, 'f1_pos': 0.41, 'f1_neu': 0.426, 'f1_neg': 0.587, 'eval_score': 0.428}
{'n': 183, 'n_estimators': 466, 'max_depth': 3, 'learning_rate': 0.06884806868387468, 'subsample': 1, 'min_split_gain': 3}
{'accuracy': 0.527, 'f1_p

In [51]:
print(asdffs)

NameError: ignored

### Gradient Boosting

In [ ]:
# With Bayesian Optimization, and GB

def objective_func_GB(args):
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    min_samples_split = args['min_samples_split']
    min_samples_leaf = args['min_samples_leaf']

    par_dict = { 'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
    print(par_dict)
    
    clf = GradientBoostingClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
         'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
#         'criterion': hp.choice('criterion',['gini', 'entropy']),
#         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
        'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
         'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1))
        }
                                
                                
                                
best_classifier_GB= fmin(objective_func_GB, space, algo=tpe.suggest, max_evals=30)
print(best_classifier_GB)

In [ ]:
# With PCA, Bayesian Optimization, and GB

def objective_func_GB(args):
    n_estimators = args['n_estimators']
    max_depth = args['max_depth']
    min_samples_split = args['min_samples_split']
    min_samples_leaf = args['min_samples_leaf']
    n = args['n']

    par_dict = { 'n': n, 'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
    print(par_dict)
    
    clf = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, min_samples_leaf=min_samples_leaf, min_samples_split=min_samples_split, n_jobs = -1)
    accuracy, f1_Positive, f1_Neutral, f1_Negative = cross_valid_PCA(obama_train_pr_tfidf,obama_train_pr_df['Sentiment'], clf, n) 
    eval_score = round(mean([accuracy, f1_Positive, f1_Negative]),3)
    eval_dict = {'accuracy': accuracy, 'f1_pos': f1_Positive, 'f1_neu': f1_Neutral, 'f1_neg': f1_Negative, 'eval_score': eval_score}

    print(eval_dict)
    print('\n')    
    return -(eval_score)

space = {'n_estimators': hp.choice('n_estimators', np.arange(20,101, step =1)), 
         'max_depth': hp.choice('max_depth', np.arange(20,101, step =1)), 
#         'criterion': hp.choice('criterion',['gini', 'entropy']),
#         'max_features': hp.choice('max_features', ['auto', 'sqrt']),
        'min_samples_split': hp.choice('min_samples_split', np.arange(2,12, step =1)),
         'min_samples_leaf' : hp.choice('min_samples_leaf',np.arange(1,12, step =1)),
        'n': hp.choice('n', np.arange(3,203, step =20))
        }
                                
                                
                                
best_classifier_RF_PCA = fmin(objective_func_RF_PCA, space, algo=tpe.suggest, max_evals=30)
print(best_classifier_RF_PCA)

In [ ]:
from imblearn.over_sampling import SMOTE


sm = SMOTE(sampling_strategy='not majority',random_state=42)
romney_train_pr_tfidf_smote, romney_train_pr_class = sm.fit_sample(romney_train_pr_tfidf, romney_train_pr_df['Sentiment'] )


sm = SMOTE(sampling_strategy='not majority',random_state=42)
obama_train_pr_tfidf_smote, obama_train_pr_class = sm.fit_sample(obama_train_pr_tfidf, obama_train_pr_df['Sentiment'])